# Create SSH Tunnels

 

## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

## Create the Experiment Slice



In [2]:
# Create a slice
slice = fablib.new_slice(name="MySlice")

# Add a node
node = slice.add_node(name="Node1", disk=100)
node.enable_docker()   
    
# Submit the slice
slice.submit();


Retry: 6, Time: 491 sec


ID,6fcf14ba-7f0c-4de2-90f0-c5725acd5812
Name,MySlice
Lease Expiration (UTC),2023-03-02 22:16:53 +0000
Lease Start (UTC),2023-03-01 22:16:54 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
fcd273f6-69a8-409a-a636-d4c73d0358d9,Node1,2,8,100,default_rocky_8,qcow2,clem-w1.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fe68:99d2,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fe68:99d2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Run a Droppy Server in the Node

Note: the Droppy server listens on port 8989 on the node.


In [3]:
node = slice.get_node('Node1')

#node.upload_directory(f'{os.environ["HOME"]}/work/docker_containers','.')
node.execute(f"curl -O https://raw.githubusercontent.com/silverwind/droppy/master/examples/docker-compose.yml ; " 
             f"docker compose up -d ;"
             f"sudo chown $(id -u):$(id -g) data config ;"
             , quiet=True, output_file=f"{node.get_name()}.log");

## Start the SSH Tunnel

- Download your custom fabric_ssh_tunnel_package.tgz tarball from the main jupyter-examples folder OR the fabric_config folder.  This tarball was created when you ran the configure environment notebook. Rerun the configure environment notebook if you do not have the  fabric_ssh_tunnel_package.tgz 
- Open the tarball
- Run the ssh_tunnel.sh script found in the untarred folder. You can either open a terminal and execute the script or you can double-click the script in a file browser. Note: double-clicking depends on your configuration or your local machine. On a Mac, double-clicking will work if you ensure the ssh_tunnel.sh script is run using Terminal.app (Linux and Windows are tbd.)
- The script will ask you for the following:
    - Target VM: your VM's username and management IP (ex: rocky@1.2.3.4)
    - Local port: Any free port on your localhost (ex. 5555) 
    - Target_port: The port on the VM that the server is listening to. (In this example, Droppy is listening to port 8989)
- After running the script, leave the terminal window open and connect to localhost:<local_port> using a client application that can connect to your VM's service.  In the example, you can connect to Droppy by using a browser to navigate to to http://localhost:5555 
- Now use your service.  
 



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()